<a href="https://colab.research.google.com/github/AakashAhuja30/Pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transform

In [2]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
    self.fc1= nn.Linear(16*7*7, num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)

    return x

In [3]:
#Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
#Hyperparameters
input_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [5]:
#Load Data
train_dataset = datasets.MNIST(root='dataset/',train = True, transform=transform.ToTensor(), download=True)
train_loader = DataLoader(dataset = train_dataset , batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train = False, transform=transform.ToTensor(), download=True)
test_loader = DataLoader(dataset = test_dataset , batch_size=batch_size,shuffle=True)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
model= CNN().to(device)
#Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [9]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print('Checking accuracy on Train data')
  else:
    print('Checking accuracy on Test data')

  num_samples =  0
  num_correct = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device = device)
      y = y.to(device = device)

      scores= model(x)
      _,pred=torch.max(scores,1)
      num_correct += (pred ==y).sum()
      num_samples += pred.size(0)

    print(f'Accuracy is :{(num_correct/num_samples)*100}')

  model.train()  

In [10]:
for epoch in range(num_epochs):
  for batch_idx, (data,target) in enumerate(train_loader):
    data = data.to(device = device)
    target = target.to(device = device)
    
    #Forward pass
    output = model(data)
    loss = criterion(output,target)

    #Backward pass
    loss.backward()

    #Gradient descent
    optimizer.step()
    optimizer.zero_grad()

  print(f'epoch{epoch+1}')
  check_accuracy(train_loader,model)
  check_accuracy(test_loader,model)

epoch1
Checking accuracy on Train data
Accuracy is :96.0
Checking accuracy on Test data
Accuracy is :96.15999603271484
epoch2
Checking accuracy on Train data
Accuracy is :97.49667358398438
Checking accuracy on Test data
Accuracy is :97.53999328613281
epoch3
Checking accuracy on Train data
Accuracy is :98.12833404541016
Checking accuracy on Test data
Accuracy is :97.93999481201172
epoch4
Checking accuracy on Train data
Accuracy is :98.36000061035156
Checking accuracy on Test data
Accuracy is :98.1199951171875
epoch5
Checking accuracy on Train data
Accuracy is :98.64666748046875
Checking accuracy on Test data
Accuracy is :98.47999572753906
epoch6
Checking accuracy on Train data
Accuracy is :98.63166809082031
Checking accuracy on Test data
Accuracy is :98.37999725341797
epoch7
Checking accuracy on Train data
Accuracy is :98.81999969482422
Checking accuracy on Test data
Accuracy is :98.35999298095703
epoch8
Checking accuracy on Train data
Accuracy is :99.03500366210938
Checking accuracy on